In [1]:
# Install required Python packages
!pip install --upgrade pip  # Always good to ensure pip is up to date
!pip install onnx onnxruntime git+https://github.com/espressif/esp-ppq.git
!pip install protobuf==3.20.2
!pip install torchvision==0.17.2
!git lfs install
!find imagenet-sample-images/ -mindepth 1 -name ".*" -exec rm -rf {} +
!pip install ultralytics


  Cloning https://github.com/espressif/esp-ppq.git to /private/var/folders/mv/96db_j8j3ng3t9pc8jczncdh0000gn/T/pip-req-build-bvkymbyw
  Running command git clone --filter=blob:none --quiet https://github.com/espressif/esp-ppq.git /private/var/folders/mv/96db_j8j3ng3t9pc8jczncdh0000gn/T/pip-req-build-bvkymbyw
  Resolved https://github.com/espressif/esp-ppq.git to commit 0dc842f97dd3945aa430d6eecb7ed98926dc9d6c
  Preparing metadata (setup.py) ... done
Updated Git hooks.
Git LFS initialized.
find: imagenet-sample-images/: No such file or directory


In [2]:
from pathlib import Path
import os
from typing import Final
from PIL import Image
from ppq.api import espdl_quantize_onnx
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from ultralytics import YOLO
import sys
sys.path.append("models/coco_detect/models")
from export_onnx import ESP_YOLO, ESP_Attention, ESP_Detect
from ultralytics.nn.modules import Attention, Detect



    ___________ ____        ____  ____  ____ 
   / ____/ ___// __ \      / __ \/ __ \/ __ \
  / __/  \__ \/ /_/ /_____/ /_/ / /_/ / / / /
 / /___ ___/ / ____/_____/ ____/ ____/ /_/ / 
/_____//____/_/         /_/   /_/    \___\_\ 


Ultralytics 8.3.130 🚀 Python-3.10.11 torch-2.2.2 CPU (Apple M3)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 64, 80, 80), (1, 80, 80, 80), (1, 64, 40, 40), (1, 80, 40, 40), (1, 64, 20, 20), (1, 80, 20, 20)) (5.4 MB)

ONNX: starting export with onnx 1.17.0 opset 13...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.5s, saved as 'yolo11n.onnx' (10.1 MB)

Export complete (0.7s)
Results saved to /Users/christophknaden/git/esp-dl-deployment
Predict:         yolo predict task=detect model=yolo11n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolo11n.onnx imgsz=640 data=/usr/src/ultralytics/ultr

In [3]:
model = YOLO('./models/coco_detect/models/yolo11n.pt')

torch_model = model.model.eval()

In [4]:
DEVICE: str = 'cpu'

BATCH_SIZE = 1
IMAGE_SIZE = 640

IMAGENET_PATH: Final[Path] = Path('./calib_images')

class ImageFolderDataset(Dataset):
    def __init__(self, image_dir: Path, transform=None):
        self.image_files = [
            image_dir / f for f in os.listdir(image_dir)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image = Image.open(self.image_files[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# Transformation (resize only — no normalization!)
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
])

# Create dataset and dataloader
cal_dataset = ImageFolderDataset(IMAGENET_PATH, transform=transform)
cal_dataloader = DataLoader(
    cal_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
)


In [6]:
# Calibration images need to be in folder calib-images
IMAGE_PATH: Final[Path] = Path('./calib_images')


for img_path in IMAGE_PATH.glob("*.jpg"):
    results = model(img_path)
    results[0].save(filename=f"./preds-calib-images/output_{img_path.name}")


image 1/1 /Users/christophknaden/git/esp-dl-deployment/calib_images/000000475732.jpg: 480x640 1 cat, 53.0ms
Speed: 1.2ms preprocess, 53.0ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/christophknaden/git/esp-dl-deployment/calib_images/000000098261.jpg: 448x640 2 birds, 47.9ms
Speed: 1.9ms preprocess, 47.9ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /Users/christophknaden/git/esp-dl-deployment/calib_images/bikes_compressed.jpg: 640x640 1 person, 9 bicycles, 76.4ms
Speed: 1.2ms preprocess, 76.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/christophknaden/git/esp-dl-deployment/calib_images/000000181542.jpg: 608x640 9 persons, 1 bicycle, 2 cars, 5 motorcycles, 1 bus, 1 truck, 63.7ms
Speed: 1.0ms preprocess, 63.7ms inference, 0.4ms postprocess per image at shape (1, 3, 608, 640)

image 1/1 /Users/christophknaden/git/esp-dl-deployment/calib_images/000000574315.jpg: 480x640 1 pers

In [6]:
# You need to execute the following command in this github project http://github.com/espressif/esp-dl in this folder models/coco_detect/models/
# python export_onnx.py --weights ./yolo11n.pt --img 640 --batch 1
# models/coco_detect/models/export_onnx.py

model = ESP_YOLO("models/coco_detect/models/yolo11n.pt")
for m in model.modules():
    if isinstance(m, Attention):
        m.forward = ESP_Attention.forward.__get__(m)
    if isinstance(m, Detect):
        m.forward = ESP_Detect.forward.__get__(m)

# Export to ONNX
model.export(format="onnx", simplify=True, opset=13, dynamic=False, imgsz=640)

Ultralytics 8.3.130 🚀 Python-3.10.11 torch-2.2.2 CPU (Apple M3)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'models/coco_detect/models/yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 64, 80, 80), (1, 80, 80, 80), (1, 64, 40, 40), (1, 80, 40, 40), (1, 64, 20, 20), (1, 80, 20, 20)) (5.4 MB)

ONNX: starting export with onnx 1.17.0 opset 13...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.6s, saved as 'models/coco_detect/models/yolo11n.onnx' (10.1 MB)

Export complete (0.9s)
Results saved to /Users/christophknaden/git/esp-dl-deployment/models/coco_detect/models
Predict:         yolo predict task=detect model=models/coco_detect/models/yolo11n.onnx imgsz=640  
Validate:        yolo val task=detect model=models/coco_detect/models/yolo11n.onnx imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


'models/coco_detect/models/yolo11n.onnx'

In [7]:
ONNX_YOLO_PATH: Final[Path] = Path('./models/coco_detect/models/yolo11n.onnx')
ESPDL_YOLO_PATH: Final[Path] = Path('./models/coco_detect/models/s3/yolo11n.espdl')
TARGET_SOC: Final[str] = 'esp32s3'
NUM_OF_BITS: Final[int] = 8


x = cal_dataset[0]
if isinstance(x, (tuple, list)):
    x = x[0]
x = x.unsqueeze(0)

def collate_fn(batch: torch.Tensor) -> torch.Tensor:
    return batch.to(DEVICE)

# make use of ESP-PPQ to quantize the ONNX computation graph, optimize it for ESP32-S3 SoC, and convert it to .espdl
quantized_model = espdl_quantize_onnx(
    onnx_import_file=ONNX_YOLO_PATH.as_posix(),
    espdl_export_file=ESPDL_YOLO_PATH.as_posix(),
    calib_dataloader=cal_dataloader,
    calib_steps=8,
    input_shape=x.shape,
    inputs=None,
    target=TARGET_SOC,
    num_of_bits=NUM_OF_BITS,
    collate_fn=collate_fn,
    dispatching_override=None,
    device=DEVICE,
    error_report=True,
    skip_export=False,
    export_test_values=True,
    verbose=1,
)

[Warning] Unexpected input value of operation /model.11/Resize, recieving "None" at its input 1
[Warning] Unexpected input value of operation /model.14/Resize, recieving "None" at its input 1
[11:08:56] PPQ Quantization Fusion Pass Running ...       Finished.
[11:08:56] PPQ Quantize Simplify Pass Running ...         Finished.
[11:08:56] PPQ Parameter Quantization Pass Running ...    Finished.
[11:08:56] PPQ Runtime Calibration Pass Running ...       

Calibration Progress(Phase 2): 100%|██████████| 8/8 [00:01<00:00,  5.15it/s]


Finished.
[11:09:00] PPQ Quantization Alignment Pass Running ...    Finished.
[11:09:00] PPQ Passive Parameter Quantization Running ... Finished.
--------- Network Snapshot ---------
Num of Op:                    [296]
Num of Quantized Op:          [296]
Num of Variable:              [499]
Num of Quantized Var:         [499]
------- Quantization Snapshot ------
Num of Quant Config:          [915]
ACTIVATED:                    [334]
OVERLAPPED:                   [369]
PASSIVE:                      [194]
FP32:                         [18]
Network Quantization Finished.


Analysing Graphwise Quantization Error(Phrase 1):: 100%|██████████| 8/8 [00:01<00:00,  5.96it/s]
Analysing Graphwise Quantization Error(Phrase 2):: 100%|██████████| 8/8 [00:02<00:00,  2.86it/s]


Layer                                        | NOISE:SIGNAL POWER RATIO 
/model.10/m/m.0/ffn/ffn.1/conv/Conv:         | ████████████████████ | 46.525%
/model.23/cv3.2/cv3.2.1/cv3.2.1.1/conv/Conv: | ████████████         | 27.680%
/model.10/m/m.0/attn/proj/conv/Conv:         | ███████████          | 25.686%
/model.23/cv3.2/cv3.2.0/cv3.2.0.0/conv/Conv: | ██████████           | 23.715%
/model.23/cv2.2/cv2.2.1/conv/Conv:           | ██████████           | 23.019%
/model.23/cv3.2/cv3.2.0/cv3.2.0.1/conv/Conv: | ██████████           | 22.853%
/model.23/cv2.2/cv2.2.0/conv/Conv:           | █████████            | 21.466%
/model.22/m.0/cv3/conv/Conv:                 | █████████            | 19.918%
/model.22/m.0/m/m.1/cv2/conv/Conv:           | ████████             | 19.583%
/model.22/cv2/conv/Conv:                     | ████████             | 19.061%
/model.23/cv3.1/cv3.1.1/cv3.1.1.1/conv/Conv: | ████████             | 18.990%
/model.22/m.0/cv2/conv/Conv:                 | ████████             |

Analysing Layerwise quantization error:: 100%|██████████| 89/89 [02:27<00:00,  1.66s/it]


Layer                                        | NOISE:SIGNAL POWER RATIO 
/model.1/conv/Conv:                          | ████████████████████ | 0.488%
/model.22/cv1/conv/Conv:                     | █████████████        | 0.312%
/model.4/cv2/conv/Conv:                      | █████████████        | 0.306%
/model.2/cv2/conv/Conv:                      | ██████████           | 0.242%
/model.0/conv/Conv:                          | █████████            | 0.219%
/model.10/cv1/conv/Conv:                     | ████████             | 0.199%
/model.9/cv2/conv/Conv:                      | ███████              | 0.170%
/model.3/conv/Conv:                          | ████                 | 0.108%
/model.4/cv1/conv/Conv:                      | ████                 | 0.090%
/model.2/cv1/conv/Conv:                      | ███                  | 0.077%
/model.16/cv1/conv/Conv:                     | ███                  | 0.072%
/model.23/cv2.0/cv2.0.0/conv/Conv:           | ██                   | 0.058%
/mo

In [9]:
from PIL import Image

foo = Image.open('./bikes.jpg')
foo.size  # (1200, 1200)

# downsize the image with an ANTIALIAS filter (gives the highest quality)
foo = foo.resize((640,640),Image.Resampling.LANCZOS)

foo.save('./bikes_compressed.jpg', quality=95)

#foo.save('./bikes_compressed.jpg', optimize=True, quality=95)